In [203]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vamshi/711pAe-vJhL.jpg
/kaggle/input/processed-data/processed.csv.csv
/kaggle/input/dataset/711gPYqqqIL.jpg
/kaggle/input/dataset/71nywfWZUwL.jpg
/kaggle/input/dataset/3.jpg
/kaggle/input/dataset/51WsuKKAVrL.jpg
/kaggle/input/newdataset/processed (10).csv
/kaggle/input/testing3/21i52HRW4L.jpg
/kaggle/input/images/1.jpg
/kaggle/input/images/2.jpg
/kaggle/input/testing2/test.csv
/kaggle/input/amazon-dataset2/train.csv
/kaggle/input/images21/61sQqAKr4L.jpg


In [204]:
pip install tensorflow==2.15

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [205]:
pip install keras_ocr

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [206]:
pip install pytesseract

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [207]:

# Ensuring TensorFlow uses GPU (default behavior)
import tensorflow as tf

# Check if TensorFlow is using GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to avoid full allocation of the GPU memory
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU is available: {gpus}")
    except RuntimeError as e:
        print(e)


Num GPUs Available:  2
GPU is available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# **Image Preprocessing**

In [222]:
import keras_ocr
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf  # For GPU support
import re
import cv2
import pytesseract
from PIL import Image
import tempfile

# Load the recognizer pipeline
pipeline = keras_ocr.pipeline.Pipeline(scale=50)

# Function to detect text using keras-ocr
def detect_text_with_keras_ocr(image):
    images = [image]
    prediction_groups = pipeline.recognize(images)
    return prediction_groups

# Function to load and detect text using keras-ocr
def process_and_detect_text(image_path):
    image = keras_ocr.tools.read(image_path)
    prediction_groups = detect_text_with_keras_ocr(image)
    return image, prediction_groups

# Function for normalization of images
def normalize_image(image):
    norm_img = np.zeros((image.shape[0], image.shape[1]))
    image = cv2.normalize(image, norm_img, 0, 255, cv2.NORM_MINMAX)
    return image

# Function to remove noise from images
def remove_noise(image):
    # Check if the image is grayscale (single channel)
    if len(image.shape) == 2:  # Grayscale
        return cv2.fastNlMeansDenoising(image, None, 10, 7, 21)
    
    elif len(image.shape) == 3:  # Colored
        return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)
    else:
        raise ValueError("Unsupported image format for denoising")

# Main preprocessing function
def preprocess_image(image):
    image = normalize_image(image)
    gray = get_grayscale(image)
    denoised = remove_noise(gray)  # Applies denoising based on image type
    binary = thresholding(denoised)
    upscaled_image = cv2.resize(binary, None, fx=6 ,fy=6, interpolation=cv2.INTER_CUBIC)
    return upscaled_image


# Function to convert image to grayscale
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Function to threshold images
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


# Weight

In [223]:
import re
import cv2
import numpy as np
import pytesseract

def find_and_crop_weight(image_path, keywords=None, non_weight_keywords=None):
    # Default weight-related keywords
    if keywords is None:
        keywords = ["kg", "gm", "g", "mg", "grams", "milligrams", "kilograms", 'G', 'KG', 'Kg', 'pounds', 'oz','lb','lbs']
    
    # Default non-weight-related keywords
    if non_weight_keywords is None:
        non_weight_keywords = ["mAh", "Ah", "A-h", "v", "W"]

    # Load the image and detect text boxes using your process_and_detect_text function
    image, prediction_groups = process_and_detect_text(image_path)
    cropped_images = []
    cropped_texts = []

    # Pattern to match a number followed by a weight unit
    weight_pattern = re.compile(r'(?<!\d)(\d+\.?\d*)\s*(' + r'|'.join(keywords) + r')(?!\w)', re.IGNORECASE)
    non_weight_pattern = re.compile(r'(?<!\d)(\d+\.?\d*)\s*(' + r'|'.join(non_weight_keywords) + r')(?!\w)', re.IGNORECASE)

    # Merge text boxes that are close to each other
    merged_text_boxes = []
    merged_box = None
    for i, (text, box) in enumerate(prediction_groups[0]):
        box = np.int0(box)
        x_min, y_min = np.min(box[:, 0]), np.min(box[:, 1])
        x_max, y_max = np.max(box[:, 0]), np.max(box[:, 1])

        if merged_box is None:
            merged_box = {"text": text, "x_min": x_min, "y_min": y_min, "x_max": x_max, "y_max": y_max}
        else:
            # Check if the current box is close to the previous box (both horizontally and vertically)
            if x_min - merged_box["x_max"] < 19 and abs(y_min - merged_box["y_min"]) < 19:
                # Merge the boxes and the text
                merged_box["text"] += " " + text
                merged_box["x_min"] = min(merged_box["x_min"], x_min)
                merged_box["y_min"] = min(merged_box["y_min"], y_min)
                merged_box["x_max"] = max(merged_box["x_max"], x_max)
                merged_box["y_max"] = max(merged_box["y_max"], y_max)
            else:
                # Save the merged box and start a new one
                merged_text_boxes.append(merged_box)
                merged_box = {"text": text, "x_min": x_min, "y_min": y_min, "x_max": x_max, "y_max": y_max}

    # Append the last merged box
    if merged_box:
        merged_text_boxes.append(merged_box)

    # Now process the merged text boxes
    for box in merged_text_boxes:
        text = box["text"]

        # Correct 'o' to 'oz' if followed by a number or 'o' with or without a space
        corrected_text = text
        # Check for non-weight units and ignore them
        if non_weight_pattern.search(corrected_text):
            continue  # Skip non-weight values such as "mAh"

        # Find weight-related text only
        weight_matches = weight_pattern.findall(corrected_text)
        if weight_matches:
            for match in weight_matches:
                # Crop the image for the matched weight text
                crop_img = image[max(box["y_min"] - 10, 0):min(box["y_max"] + 10, image.shape[0]),
                                 max(box["x_min"] - 10, 0):min(box["x_max"] + 10, image.shape[1])]
                preprocessed_image = preprocess_image(crop_img)

                # Count black and white pixels for inversion decision
                black_pixels = cv2.countNonZero(cv2.bitwise_not(preprocessed_image))
                white_pixels = cv2.countNonZero(preprocessed_image)
                if black_pixels > white_pixels:
                    preprocessed_image = cv2.bitwise_not(preprocessed_image)

                # Perform OCR on the preprocessed image
                custom_config = r'--oem 3 --psm 6'
                detected_text = pytesseract.image_to_string(preprocessed_image, config=custom_config).strip()

                # Check if detected_text is valid and non-empty
                if detected_text:
                    cropped_images.append(preprocessed_image)
                    cropped_texts.append(detected_text)

    return cropped_images, cropped_texts


# Height,Width,Depth

In [224]:
import re
import cv2
import numpy as np
import pytesseract

def find_and_crop_height(image_path, keywords=None):
    if keywords is None:
        keywords = ["cm", "mm", "meters", "feet", "inches", "ft", "inch", "m", "M", 'CM', 'MM', 'IN', 'INCH', 'meter', 'ft', 'Feet']

    # Load the image and detect text boxes using your `process_and_detect_text` function
    image, prediction_groups = process_and_detect_text(image_path)
    cropped_images = []
    cropped_texts = []

    # Pattern to match a number followed by a height unit
    pattern = re.compile(r'(?<!\d)(\d+\.?\d*)\s*(' + r'|'.join(keywords) + r')(?!\w)', re.IGNORECASE)

    # Merge text boxes that are close to each other
    merged_text_boxes = []
    merged_box = None
    for i, (text, box) in enumerate(prediction_groups[0]):
        box = np.int0(box)
        x_min, y_min = np.min(box[:, 0]), np.min(box[:, 1])
        x_max, y_max = np.max(box[:, 0]), np.max(box[:, 1])

        if merged_box is None:
            merged_box = {"text": text, "x_min": x_min, "y_min": y_min, "x_max": x_max, "y_max": y_max}
        else:
            # Check if the current box is close to the previous box (both horizontally and vertically)
            if x_min - merged_box["x_max"] < 19 and abs(y_min - merged_box["y_min"]) < 19:
                # Merge the boxes and the text
                merged_box["text"] += " " + text
                merged_box["x_min"] = min(merged_box["x_min"], x_min)
                merged_box["y_min"] = min(merged_box["y_min"], y_min)
                merged_box["x_max"] = max(merged_box["x_max"], x_max)
                merged_box["y_max"] = max(merged_box["y_max"], y_max)
            else:
                # Save the merged box and start a new one
                merged_text_boxes.append(merged_box)
                merged_box = {"text": text, "x_min": x_min, "y_min": y_min, "x_max": x_max, "y_max": y_max}

    # Append the last merged box
    if merged_box:
        merged_text_boxes.append(merged_box)
    for box in merged_text_boxes:
        text = box["text"]

        # Correct 'o' to 'oz' if followed by a number or 'o' with or without a space
        corrected_text = re.sub(r'(\d+\.?\d*)\s*o\b', r'\1 oz', text, flags=re.IGNORECASE)
        
        # Handle cases where 'o' might be alone or split from 'oz'
        corrected_text = re.sub(r'(\d+\.?\d*)\s*o\s*', r'\1 oz ', corrected_text, flags=re.IGNORECASE)
        
        if pattern.search(corrected_text):
            crop_img = image[max(box["y_min"] - 10, 0):min(box["y_max"] + 10, image.shape[0]),
                             max(box["x_min"] - 10, 0):min(box["x_max"] + 10, image.shape[1])]
            preprocessed_image = preprocess_image(crop_img)

            # Count black and white pixels for inversion decision
            black_pixels = cv2.countNonZero(cv2.bitwise_not(preprocessed_image))
            white_pixels = cv2.countNonZero(preprocessed_image)
            if black_pixels > white_pixels:
                preprocessed_image = cv2.bitwise_not(preprocessed_image)

            # Perform OCR on the preprocessed image
            custom_config = r'--oem 3 --psm 6'
            detected_text = pytesseract.image_to_string(preprocessed_image, config=custom_config).strip()

            # Check if detected_text is valid and non-empty
            if detected_text:
                cropped_images.append(preprocessed_image)
                cropped_texts.append(detected_text)

    return cropped_images, cropped_texts
    print('done')


# Voltage

In [225]:
import re
import cv2
import numpy as np
import pytesseract

def find_and_crop_voltage(image_path, keywords=None):
    if keywords is None:
        keywords = ["V", "volts", "Volts", "v", "Volt"]

    image, prediction_groups = process_and_detect_text(image_path)
    cropped_images = []
    cropped_texts = []

    # Pattern to match a number followed by a voltage unit
    pattern = re.compile(r'(?<!\d)(\d+\.?\d*)\s*(' + r'|'.join(keywords) + r')\b', re.IGNORECASE)
    merged_text_boxes = []
    merged_box = None
    for i, (text, box) in enumerate(prediction_groups[0]):
        box = np.int0(box)
        x_min, y_min = np.min(box[:, 0]), np.min(box[:, 1])
        x_max, y_max = np.max(box[:, 0]), np.max(box[:, 1])

        if merged_box is None:
            merged_box = {"text": text, "x_min": x_min, "y_min": y_min, "x_max": x_max, "y_max": y_max}
        else:
            # Check if the current box is close to the previous box (both horizontally and vertically)
            if x_min - merged_box["x_max"] < 19 and abs(y_min - merged_box["y_min"]) < 19:
                # Merge the boxes and the text
                merged_box["text"] += " " + text
                merged_box["x_min"] = min(merged_box["x_min"], x_min)
                merged_box["y_min"] = min(merged_box["y_min"], y_min)
                merged_box["x_max"] = max(merged_box["x_max"], x_max)
                merged_box["y_max"] = max(merged_box["y_max"], y_max)
            else:
                # Save the merged box and start a new one
                merged_text_boxes.append(merged_box)
                merged_box = {"text": text, "x_min": x_min, "y_min": y_min, "x_max": x_max, "y_max": y_max}

    # Append the last merged box
    if merged_box:
        merged_text_boxes.append(merged_box)
    for box in merged_text_boxes:
        text = box["text"]

        # Correct 'o' to 'oz' if followed by a number or 'o' with or without a space
        corrected_text = re.sub(r'(\d+\.?\d*)\s*o\b', r'\1 oz', text, flags=re.IGNORECASE)
        
        # Handle cases where 'o' might be alone or split from 'oz'
        corrected_text = re.sub(r'(\d+\.?\d*)\s*o\s*', r'\1 oz ', corrected_text, flags=re.IGNORECASE)
        
        if pattern.search(corrected_text):
            crop_img = image[max(box["y_min"] - 10, 0):min(box["y_max"] + 10, image.shape[0]),
                             max(box["x_min"] - 10, 0):min(box["x_max"] + 10, image.shape[1])]
            preprocessed_image = preprocess_image(crop_img)

            # Count black and white pixels for inversion decision
            black_pixels = cv2.countNonZero(cv2.bitwise_not(preprocessed_image))
            white_pixels = cv2.countNonZero(preprocessed_image)
            if black_pixels > white_pixels:
                preprocessed_image = cv2.bitwise_not(preprocessed_image)

            # Perform OCR on the preprocessed image
            custom_config = r'--oem 3 --psm 6'
            detected_text = pytesseract.image_to_string(preprocessed_image, config=custom_config).strip()

            # Check if detected_text is valid and non-empty
            if detected_text:
                cropped_images.append(preprocessed_image)
                cropped_texts.append(detected_text)

    return cropped_images, cropped_texts
    print('done')



# Wattage

In [226]:
import re
import cv2
import numpy as np
import pytesseract

def find_and_crop_wattage(image_path, keywords=None):
    if keywords is None:
        keywords = [
    "W", "w", "kW", "MW", "GW", "Wh", "kWh", "MWh", "GWh", 
    "VA", "kVA", "MVA", "ow", "oW"  # Extend as needed
]

    # Ensure the image is preprocessed and OCR pipeline is applied
    image, prediction_groups = process_and_detect_text(image_path)
    cropped_images = []
    cropped_texts = []

    # Pattern to match a number followed by a wattage unit
    pattern = re.compile(r'(?<!\d)(\d+\.?\d*)\s*(' + r'|'.join(keywords) + r')\b', re.IGNORECASE)

    merged_text_boxes = []
    merged_box = None
    for i, (text, box) in enumerate(prediction_groups[0]):
        box = np.int0(box)
        x_min, y_min = np.min(box[:, 0]), np.min(box[:, 1])
        x_max, y_max = np.max(box[:, 0]), np.max(box[:, 1])

        if merged_box is None:
            merged_box = {"text": text, "x_min": x_min, "y_min": y_min, "x_max": x_max, "y_max": y_max}
        else:
            # Check if the current box is close to the previous box (both horizontally and vertically)
            if x_min - merged_box["x_max"] < 19 and abs(y_min - merged_box["y_min"]) < 19:
                # Merge the boxes and the text
                merged_box["text"] += " " + text
                merged_box["x_min"] = min(merged_box["x_min"], x_min)
                merged_box["y_min"] = min(merged_box["y_min"], y_min)
                merged_box["x_max"] = max(merged_box["x_max"], x_max)
                merged_box["y_max"] = max(merged_box["y_max"], y_max)
            else:
                # Save the merged box and start a new one
                merged_text_boxes.append(merged_box)
                merged_box = {"text": text, "x_min": x_min, "y_min": y_min, "x_max": x_max, "y_max": y_max}

    # Append the last merged box
    if merged_box:
        merged_text_boxes.append(merged_box)

    for box in merged_text_boxes:
        text = box["text"]

        if pattern.search(text):
            crop_img = image[max(box["y_min"] - 10, 0):min(box["y_max"] + 10, image.shape[0]),
                             max(box["x_min"] - 10, 0):min(box["x_max"] + 10, image.shape[1])]
            preprocessed_image = preprocess_image(crop_img)
            # Count black and white pixels for inversion decision
            black_pixels = cv2.countNonZero(cv2.bitwise_not(preprocessed_image))
            white_pixels = cv2.countNonZero(preprocessed_image)
            if black_pixels > white_pixels:
                preprocessed_image = cv2.bitwise_not(preprocessed_image)

            # Perform OCR on the preprocessed image
            custom_config = r'--oem 3 --psm 6'
            detected_text = pytesseract.image_to_string(preprocessed_image, config=custom_config).strip()

            # Check if detected_text is valid and non-empty
            if detected_text:
                cropped_images.append(preprocessed_image)
                cropped_texts.append(detected_text)

    return cropped_images, cropped_texts

    print('done')


# Main Function

In [266]:
import pandas as pd
import re

# Define a mapping of unit abbreviations to their full forms
unit_mapping = {
    'kg': 'kilogram', 'gm': 'gram', 'g': 'gram', 'mg': 'milligram',
    'grams': 'gram', 'milligrams': 'milligram', 'kilograms': 'kilogram',
    'G': 'gram', 'KG': 'kilogram', 'Kg': 'kilogram', 'lb': 'pound','lbs': 'pound',
    'cm': 'centimeter', 'm': 'meter', 'millimeters': 'millimeter',
    'mm': 'millimeter', 'meters': 'meter', 'CM': 'centimeter', 'M': 'meter', 'MM': 'millimeter',
    'V': 'volt', 'volts': 'volt', 'Volt': 'volt', 'v': 'volt',
    'W': 'watt', 'watts': 'watt', 'Watts': 'watt', 'w': 'watt', 'Watt': 'watt',
    'KGS':'kilogram'
}

def call_heuristic(entity, text):
    corrected_text = []
    if entity in ['item_weight', 'maximum_weight_recommendation']:
        corrected_text = heuristic_correction_weight(text)

    elif entity in ['height', 'depth', 'width']:
        corrected_text = heuristic_correction_height(text)

    elif entity in ['voltage']:
        corrected_text = heuristic_correction_voltage(text)

    elif entity in ['wattage']:
        corrected_text = heuristic_correction_watts(text)
        
    return corrected_text    

def extract_numbers_and_units(cropped_texts, keywords, entity):
    # Regex pattern to extract numbers and units with optional spaces between
    pattern = re.compile(r'(\d+\.?\d*)\s*(' + '|'.join(keywords) + r')\b', re.IGNORECASE)
    extracted_info = []
    
    for text in cropped_texts:
        # Apply the heuristic correction based on entity type
        corrected_text = call_heuristic(entity, text)
        
        matches = pattern.findall(corrected_text)  # Find all matches
        
        for match in matches:
            number, unit = match
            full_unit = unit_mapping.get(unit.lower(), unit)  # Get the full form of the unit
            extracted_info.append(f"{number} {full_unit}")
        
        # If the entity is 'voltage' or 'wattage', call extract_value_and_unit
        if entity in ['voltage', 'wattage']:
            extracted_info = [extract_value_and_unit(info, entity) for info in extracted_info]
    
    print(extracted_info)
    return extracted_info

# Main function to process each row
def process_row(row):
    image_path = row['image_link']
    entity = row['entity_name'].lower()
    extracted_info = []

    cropped_images, cropped_texts = [], []

    if entity in ['item_weight', 'maximum_weight_recommendation']:
        cropped_images, cropped_texts = find_and_crop_weight(image_path)
        extracted_info = extract_numbers_and_units(cropped_texts, ["kg", "gm", "g", "mg", "grams", "milligrams", "kilograms", 'G', 'KG', 'Kg', 'pounds', 'oz','lb','lbs','KGS'], entity)

    elif entity in ['height', 'depth', 'width']:
        cropped_images, cropped_texts = find_and_crop_height(image_path)
        extracted_info = extract_numbers_and_units(cropped_texts, ["cm", "m", "millimeters", "mm", "meters", "CM", "M", "MM"], entity)

    elif entity in ['voltage']:
        cropped_images, cropped_texts = find_and_crop_voltage(image_path)
        extracted_info = extract_numbers_and_units(cropped_texts, ["V", "volts", "Volts", "v", "Volt"], entity)

    elif entity in ['wattage']:
        cropped_images, cropped_texts = find_and_crop_wattage(image_path)
        extracted_info = extract_numbers_and_units(cropped_texts, ["W", "w", "kW", "MW", "GW", "Wh", "kWh", "MWh", "GWh", "VA", "kVA", "MVA"], entity)

    # Print the text extracted from cropped images
    print(f"Extracted information for {image_path}:")

    for i, info in enumerate(extracted_info):
        print(f"  Extracted Info {i+1}: {info}")

    # Join all extracted info into a single string separated by commas
    return ', '.join(extracted_info)


In [228]:
import re

def heuristic_correction_weight(predicted_str):
    # Define a list of valid units including 'oz'
    valid_units = ['mg', 'g', 'kg', 'A', 'mA', 'KG', 'oz','lb','lbs']  # Extend this list as needed

    # Step 1: Remove any non-alphabetic and non-numeric characters at the end, except valid units or numbers
    trimmed_text = re.sub(r'[^\w\s]+$', '', predicted_str)

    # Step 2: If special symbols like ¢ are detected, replace them with 'g' (assuming ¢ is mistaken for 'g')
    corrected_text = re.sub(r'[¢]', 'g', trimmed_text)

    # Step 3: Extract numeric part and unit part using regex
    matches = re.findall(r'([0-9\.]+)\s*([a-zA-Z]*)', corrected_text)

    corrected_str = ""
    for value, unit in matches:
        corrected_value = value

        # Step 4: Check if the number ends with '9', '8', or '6'
        if re.search(r'[9]$', corrected_value):
            # Remove the trailing '9', '8', or '6' and append 'g'
            corrected_value = re.sub(r'[986]$', '', corrected_value)  # Remove last digit
            corrected_unit = 'g'  # Automatically append 'g'
        else:
            # If unit is close to a valid unit, replace it with the closest match
            corrected_unit = closest_unit(unit, valid_units) if unit else 'g'  # Assume 'g' if no unit is present
        
        # Special case for 'o' being mistaken for 'oz'
        if corrected_unit == '' and re.search(r'(\d+\.?\d*)\s*o\b', predicted_str):
            corrected_unit = 'oz'
        
        corrected_str += f"{corrected_value}{corrected_unit}, "

    return corrected_str.strip(', ')

# Function to find the closest unit using Levenshtein distance
def closest_unit(predicted_unit, valid_units):
    closest = min(valid_units, key=lambda x: levenshtein_distance(predicted_unit, x))
    return closest

# Levenshtein distance for measuring similarity between units
def levenshtein_distance(a, b):
    m, n = len(a), len(b)
    dp = [[0] * (n+1) for _ in range(m+1)]

    for i in range(m+1):
        for j in range(n+1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif a[i-1] == b[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])

    return dp[m][n]


In [234]:
import re
import numpy as np

# Define valid height units
valid_units = ['cm', 'mm', 'meters', 'feet', 'ft', 'inch', 'm', 'in','MM']  # Extend as needed

def closest_unit(unit, valid_units):
    """Find the closest valid unit to the given unit using Levenshtein distance."""
    if not unit:
        return 'cm'  # Default to 'cm' if no unit provided

    closest = min(valid_units, key=lambda x: levenshtein_distance(unit, x))
    return closest

def levenshtein_distance(a, b):
    """Calculate Levenshtein distance between two strings."""
    m, n = len(a), len(b)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif a[i - 1] == b[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

    return dp[m][n]

def heuristic_correction_height(predicted_str):
    # Remove non-alphabetic and non-numeric characters at the end
    trimmed_text = re.sub(r'[^\w\s\-]+$', '', predicted_str)

    # Special symbols correction
    corrected_text = re.sub(r'[¢]', 'cm', trimmed_text)  # Assuming ¢ could be a mistake for cm

    # Extract ranges and individual values
    matches = re.findall(r'([0-9\.]+)\s*[-]?\s*([0-9\.]*)\s*([a-zA-Z]*)', corrected_text)

    corrected_str = ""
    for value1, value2, unit in matches:
        if value2:  # If there's a second value, it's a range
            corrected_value1 = value1
            corrected_value2 = value2
            corrected_unit = closest_unit(unit, valid_units) if unit else 'cm'  # Default to 'cm'
            corrected_str += f"{corrected_value1} {corrected_unit} - {corrected_value2} {corrected_unit}, "
        else:  # Single value
            corrected_value = value1
            corrected_unit = closest_unit(unit, valid_units) if unit else 'cm'  # Default to 'cm'
            corrected_str += f"{corrected_value} {corrected_unit}, "

    return corrected_str.strip(', ')


In [230]:
import re
import numpy as np

# Define valid voltage units
valid_units = [
    "Volt", "Volts", "V", "v", "mV", "kV", "MV", "µV", 
    "VDC", "VAC", "DCV", "ACV"  # Extend as needed
]

def closest_unit(unit, valid_units):
    """Find the closest valid unit to the given unit using Levenshtein distance."""
    if not unit:
        return 'v'  # Default to 'v' if no unit provided

    # If the unit is 'w', return an empty string (no conversion)
    if unit.lower() == 'w':
        return ''

    # Otherwise, find the closest valid unit using Levenshtein distance
    closest = min(valid_units, key=lambda x: levenshtein_distance(unit, x))
    return closest

def levenshtein_distance(a, b):
    """Calculate Levenshtein distance between two strings."""
    m, n = len(a), len(b)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif a[i - 1] == b[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

    return dp[m][n]

def heuristic_correction_voltage(predicted_str):
    # Remove non-alphabetic and non-numeric characters at the end
    trimmed_text = re.sub(r'[^\w\s\-]+$', '', predicted_str)

    # Special symbols correction
    corrected_text = re.sub(r'[¢]', 'v', trimmed_text)  # Assuming ¢ could be a mistake for v
    
    # Extract ranges and individual values
    matches = re.findall(r'([0-9\.]+)\s*[-]?\s*([0-9\.]*)\s*([a-zA-Z]*)', corrected_text)

    corrected_str = ""
    for value1, value2, unit in matches:
        if value2:  # If there's a second value, it's a range
            corrected_value1 = value1
            corrected_value2 = value2
            corrected_unit = closest_unit(unit, valid_units) if unit else 'v'  # Default to 'v'
            if corrected_unit:  # Only append if the unit isn't empty (for 'w')
                corrected_str += f"{corrected_value1} {corrected_unit} - {corrected_value2} {corrected_unit}, "
            else:  # If the unit is 'w', only add the values without the unit
                corrected_str += f"{corrected_value1} - {corrected_value2}, "
        else:  # Single value
            corrected_value = value1
            corrected_unit = closest_unit(unit, valid_units) if unit else 'v'  # Default to 'v'
            if corrected_unit:  # Only append if the unit isn't empty (for 'w')
                corrected_str += f"{corrected_value} {corrected_unit}, "
            else:  # If the unit is 'w', only add the value without the unit
                corrected_str += f"{corrected_value}, "

    return corrected_str.strip(', ')



In [231]:
import re
import numpy as np

# Define valid power units
valid_power_units = [
    "W", "w", "kW", "MW", "GW", "Wh", "kWh", "MWh", "GWh", 
    "VA", "kVA", "MVA"  # Extend as needed
]

def closest_power_unit(unit, valid_power_units):
    """Find the closest valid power unit to the given unit using Levenshtein distance."""
    if not unit:
        return 'W'  # Default to 'W' if no unit provided

    # Otherwise, find the closest valid unit using Levenshtein distance
    closest = min(valid_power_units, key=lambda x: levenshtein_distance(unit, x))
    return closest

def levenshtein_distance(a, b):
    """Calculate Levenshtein distance between two strings."""
    m, n = len(a), len(b)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif a[i - 1] == b[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

    return dp[m][n]

def heuristic_correction_watts(predicted_str):
    # Remove non-alphabetic and non-numeric characters at the end
    trimmed_text = re.sub(r'[^\w\s\-]+$', '', predicted_str)

    # Special symbols correction
    corrected_text = re.sub(r'[¢]', 'W', trimmed_text)  # Assuming ¢ could be a mistake for W
    
    # Extract ranges and individual values
    matches = re.findall(r'([0-9\.]+)\s*[-]?\s*([0-9\.]*)\s*([a-zA-Z]*)', corrected_text)

    corrected_str = ""
    for value1, value2, unit in matches:
        if value2:  # If there's a second value, it's a range
            corrected_value1 = value1
            corrected_value2 = value2
            corrected_unit = closest_power_unit(unit, valid_power_units) if unit else 'W'  # Default to 'W'
            corrected_str += f"{corrected_value1} {corrected_unit} - {corrected_value2} {corrected_unit}, "
        else:  # Single value
            corrected_value = value1
            corrected_unit = closest_power_unit(unit, valid_power_units) if unit else 'W'  # Default to 'W'
            corrected_str += f"{corrected_value} {corrected_unit}, "

    return corrected_str.strip(', ')


In [232]:
import re

# Function to map the extracted unit to the desired form based on entity_type
def map_unit(value, unit, entity_type):
    if entity_type == 'voltage' and unit == 'V':  # Voltage transformation
        if value < 1:
            return f"{value * 1000} millivolt"
        elif value >= 1000:
            return f"{value / 1000} kilovolt"
        else:
            return f"{value} volt"
    elif entity_type == 'wattage' and unit == 'W':  # Wattage transformation
        if value >= 1000:
            return f"{value / 1000} kilowatt"
        else:
            return f"{value} watt"
    # Default mapping if the entity_type does not match
    return f"{value} {unit}"

# Function to extract and map the numeric value and unit
def extract_value_and_unit(text, entity_type):
    # Regular expression to capture numeric value and unit (including values without space like '54W')
    matches = re.findall(r'(\d+\.?\d*)\s*([VWvw]|[Vv]olt[s]?|[Ww]att[s]?)', text)

    if matches:
        # Filter matches based on the entity_type priority
        for value, unit in matches:
            # Handle different variations of voltage units
            if entity_type == 'voltage' and unit.lower() in ['v', 'volt', 'volts']:
                return map_unit(float(value), 'V', entity_type)  # Use standardized 'V' for voltage

            # Handle different variations of wattage units
            elif entity_type == 'wattage' and unit.lower() in ['w', 'watt', 'watts']:
                return map_unit(float(value), 'W', entity_type)  # Use standardized 'W' for wattage

        # If no match is found based on entity_type, return the first match found
        value, unit = matches[0]
        if unit.lower() in ['v', 'volt', 'volts']:
            return map_unit(float(value), 'V', entity_type)
        elif unit.lower() in ['w', 'watt', 'watts']:
            return map_unit(float(value), 'W', entity_type)

    else:
        return "No valid value and unit found"


# Example usage
data = {'entity_type': 'wattage'}
text1 = "The device operates at 1.1V / 5W"
text2 = "The voltage is 500 V"
text3 = "Power consumption is 1500W"

print(extract_value_and_unit(text1, data['entity_type']))  # Output: "5.0 watt"
print(extract_value_and_unit(text2, 'voltage'))  # Output: "500 volt"
print(extract_value_and_unit(text3, 'wattage'))  # Output: "1.5 kilowatt"

5.0 watt
500.0 volt
1.5 kilowatt


In [287]:
# Test examples:
examples = [
    "1400MG", "1400nc", "20060", "A2009", "1009w", "5mw", "35INw",'65w/20v', "169q", "0.592kg", "2k",'2009w;%^','abcd fghh sqwfg 900g dqwfe4 25g dfeve 5kg','Weight: 26¢','3o2','5cg','40cm | 30cm'
]

for example in examples:
    corrected = call_heuristic('width',example)
    print(f"Predicted: {example} -> Corrected: {corrected}")

Predicted: 1400MG -> Corrected: 1400 cm
Predicted: 1400nc -> Corrected: 1400 cm
Predicted: 20060 -> Corrected: 20060 cm
Predicted: A2009 -> Corrected: 2009 cm
Predicted: 1009w -> Corrected: 1009 m
Predicted: 5mw -> Corrected: 5 mm
Predicted: 35INw -> Corrected: 35 cm
Predicted: 65w/20v -> Corrected: 65 m, 20 m
Predicted: 169q -> Corrected: 169 m
Predicted: 0.592kg -> Corrected: 0.592 cm
Predicted: 2k -> Corrected: 2 m
Predicted: 2009w;%^ -> Corrected: 2009 m
Predicted: abcd fghh sqwfg 900g dqwfe4 25g dfeve 5kg -> Corrected: 900 m, 4 m - 25 m, 5 cm
Predicted: Weight: 26¢ -> Corrected: 26 cm
Predicted: 3o2 -> Corrected: 3 m, 2 cm
Predicted: 5cg -> Corrected: 5 cm
Predicted: 40cm | 30cm -> Corrected: 40 cm, 30 cm


In [288]:
# Load the dataset (assuming the dataset is in CSV format)
df = pd.read_csv('/kaggle/input/testing2/test.csv')

# Taking a random sample of 100 rows for testing

# Applying the process_row function to each row and assigning the result to a new column
df['detected_text2'] = df.apply(lambda row: process_row(row), axis=1)

# Display the first few rows to check results
df.head()


1/1 [==============================] - 0s 40ms/step
[]
Extracted information for https://m.media-amazon.com/images/I/110EibNyclL.jpg:
1/1 [==============================] - 0s 24ms/step


/tmp/ipykernel_36/4017202907.py:22: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


1/1 [==============================] - 0s 42ms/step
[]
Extracted information for https://m.media-amazon.com/images/I/11TU2clswzL.jpg:
1/1 [==============================] - 0s 44ms/step
[]
Extracted information for https://m.media-amazon.com/images/I/11TU2clswzL.jpg:
1/1 [==============================] - 0s 47ms/step
[]
Extracted information for https://m.media-amazon.com/images/I/11TU2clswzL.jpg:
1/1 [==============================] - 0s 45ms/step
[]
Extracted information for https://m.media-amazon.com/images/I/11gHj8dhhrL.jpg:
1/1 [==============================] - 0s 45ms/step
[]
Extracted information for https://m.media-amazon.com/images/I/11gHj8dhhrL.jpg:
1/1 [==============================] - 0s 54ms/step
[]
Extracted information for https://m.media-amazon.com/images/I/11gHj8dhhrL.jpg:
1/1 [==============================] - 0s 36ms/step
[]
Extracted information for https://m.media-amazon.com/images/I/11lshEUmCrL.jpg:
1/1 [==============================] - 0s 39ms/step
['40 centi

/tmp/ipykernel_36/4017202907.py:22: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


['40 centimeter', '30 centimeter', '185 centimeter']
Extracted information for https://m.media-amazon.com/images/I/21-LmSmehZL.jpg:
  Extracted Info 1: 40 centimeter
  Extracted Info 2: 30 centimeter
  Extracted Info 3: 185 centimeter
1/1 [==============================] - 0s 41ms/step
[]
Extracted information for https://m.media-amazon.com/images/I/213oP6n7jtL.jpg:


...

[]
Extracted information for https://m.media-amazon.com/images/I/21cLufe8Y5L.jpg:
1/1 [==============================] - 0s 45ms/step


/tmp/ipykernel_36/4017202907.py:22: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


['26 centimeter']
Extracted information for https://m.media-amazon.com/images/I/21d6Dtc94mL.jpg:
  Extracted Info 1: 26 centimeter


/tmp/ipykernel_36/3066048154.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['detected_text2'] = df_subset.apply(lambda row: process_row(row), axis=1)


,index,image_link,group_id,entity_name,detected_text2
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,


In [289]:
 df.to_csv('/kaggle/working/processed.csv', index=False)

In [290]:
import pandas as pd
import numpy as np

# Function to process detected_text2 based on entity_name
def process_detected_text(entity_name, detected_text2):
    if pd.isnull(detected_text2):
        return np.nan  # Keep NaN for now to handle later
    
    values = detected_text2.split(',')  # Assuming values are comma-separated
    if entity_name == 'height':
        return values[0] if len(values) > 0 else np.nan
    elif entity_name == 'width':
        return values[1] if len(values) > 1 else np.nan
    elif entity_name == 'depth':
        return values[2] if len(values) > 2 else np.nan
    else:
        return values[0] if len(values) > 0 else np.nan

# Apply the processing function to each row
df['processed_text'] = df.apply(lambda row: process_detected_text(row['entity_name'], row['detected_text2']), axis=1)

# Compute mode for each entity_name
modes = df.groupby('entity_name')['processed_text'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

# Function to fill null values with mode
def fill_null_with_mode(row):
    if pd.isnull(row['processed_text']):
        return modes[row['entity_name']] if row['entity_name'] in modes else np.nan
    return row['processed_text']

# Apply the fill_null_with_mode function
df['processed_text'] = df.apply(fill_null_with_mode, axis=1)

# Replace detected_text2 with processed_text
df['detected_text2'] = df['processed_text']

# Remove the processed_text column
df.drop(columns=['processed_text'], inplace=True)

# Now, let's create df_final by copying the required columns
df_final = pd.DataFrame()
df_final['index'] = df['index']  # Copy the 'index' column
df_final['entity_value'] = df['detected_text2']  # Rename 'processed_text' to 'entity_value'

# Save to a new CSV file
output_file = '/kaggle/working/processed_output.csv'  # Corrected path
df_final.to_csv(output_file, index=False)

# Show the first few rows of df_final
df_final.head()


/tmp/ipykernel_36/3356738012.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['processed_text'] = df_subset.apply(lambda row: process_detected_text(row['entity_name'], row['detected_text2']), axis=1)
/tmp/ipykernel_36/3356738012.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['processed_text'] = df_subset.apply(fill_null_with_mode, axis=1)
/tmp/ipykernel_36/3356738012.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

,index,entity_value
0,0,
1,1,30 centimeter
2,2,
3,3,10 centimeter
4,4,10 centimeter
